<a href="https://colab.research.google.com/github/pe44enka/DashBoard/blob/master/DashBoard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **US Domestic Airline Flights Performance Dashboard**

![](https://img.freepik.com/premium-vector/concept-flight-delay-cancel-change-plans-tired-perplexed-upset-flight-delay-girl-sitting-luggage-waiting-departure-airport_165932-74.jpg?w=2000)

### Objectives
As a data analyst, you have been given a task to monitor and report US domestic airline flights performance to analyze the performance and improve flight reliability thereby improving customer reliability.


---
### Goal of the project
Create a **dashboard** with 2 possible options of reports for particular year:

* **Yearly airline performance report:**
  * *bar chart:* monthly number of flights under different cancellation categories
  * *line chart:* average monthly flight time (minutes) by reporting airline
  * *pie cahrt:* percentage of diverted airport landings per reporting airline
  * *choropleth map:* number of flights flying from each state 
  * *tree chart:* number of flights flying to each state from each reporting airline 
* **Yearly average flight delay statistics:** monthly average delay by reporting airline for the given year
  * carrier delay
  * weather delay
  * national air system delay
  * security delay 
  * late aircraft delay 

**Note:** Year range is between 2005 and 2020


**Expected Layout:**
![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/Module%205/images/Layout.png)



### Techniques
In this project I will use:
* **Data manipulation:** pandas
* **Chart plotting:** plotly
* **Dashboard building:** dash
* **Dashboard run:** ngrok

---

## Libraries





First of all, let's install and import all libraries we need.

---



In [1]:
# Install Dash
!pip install dash==0.31.1  # The core dash backend
!pip install dash-html-components==0.13.2  # HTML components
!pip install dash-core-components==0.39.0  # Supercharged components
!pip install dash --upgrade

# Install ngrok wrapper
!pip install pyngrok 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached dash-0.31.1-py3-none-any.whl
  Attempting uninstall: dash
    Found existing installation: dash 2.8.1
    Uninstalling dash-2.8.1:
      Successfully uninstalled dash-2.8.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached dash_html_components-0.13.2-py3-none-any.whl
  Attempting uninstall: dash-html-components
    Found existing installation: dash-html-components 2.0.0
    Uninstalling dash-html-components-2.0.0:
      Successfully uninstalled dash-html-components-2.0.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached dash_core_components-0.39.0-py3-none-any.whl
  Attempting uninstall: dash-core-components
    Found existing installation: dash-core-components 2.0.0
    Uninstalling dash-core-components-2.0.0:
      Successfully uninstalled dash-c

In [57]:
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output, State
from dash import no_update
import plotly.graph_objects as go
import plotly.express as px
from pyngrok import ngrok


## ngrok tunnel



To be able to run and share Dash Web Application using a Public URL I will use secure tunneling solution called `ngrok`. To launch it you need `Authtoken` you get after registration of your free account.

---



In [3]:
!ngrok authtoken # add your authtoken here

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [56]:
url = ngrok.connect(8050)
print('Public link: ', url.public_url)

Public link:  http://209a-34-75-213-7.ngrok.io




---

I will use this link later (after running the application) to see the result.

---



## Application Script


**Requirements:**
* two dropdown menus: 
  * report type 
  * year
* each dropdown: outer division with two inner divisions:
    * information about the dropdown
    * dropdown
* layout for adding graphs
* callback function to compute data, create graph and return to the layout

---

In [55]:
### Save file with Dash app on the Google Colab machine
#%%writefile airline_report.py

# create dash app
app = dash.Dash(__name__)


# clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True

# read airline data inro pandas dataframe
airline_df = pd.read_csv('https://raw.githubusercontent.com/pe44enka/DashBoard/master/data/DelayedFlights.cvs',
                         dtype={'Div1Airport': str, 'Div1TailNum': str, 
                                'Div2Airport': str, 'Div2TailNum': str})

# list of years
year_list = [i for i in range(2005, 2021)]


'''
Compute graph data for creating yearly airline performance report.
Function that takes airline data as input and create 5 dataframes based on the grouping condition to be used for plotting charts and graphs.
Arguments: 
  df: airline dataframe
Returns:
  computed dataframes for cancelation category count, average flight time by reporting airline, deverted airport landing, source state count, destination state count
'''
def compute_data_choice_1(df):
  # cancelation category count
  bar_data = df.groupby(['Month', 'CancellationCode'])['Flights'].sum().reset_index()
  # average flight time by reporting airline
  line_data = df.groupby(['Month', 'Reporting_Airline'])['AirTime'].mean().reset_index()
  # diverted airport landing
  div_data = df[df['DivAirportLandings']!=0.0]
  # source state count
  map_data = df.groupby(['OriginState'])['Flights'].sum().reset_index()
  # destination state count
  tree_data = df.groupby(['DestState', 'Reporting_Airline'])['Flights'].sum().reset_index()
  return bar_data, line_data, div_data, map_data, tree_data


'''
Compute graph data for creating Yearly average flight delay statistics.
Function that takes airline data as input and create 5 dataframes based on grouping condition to be used for plotting chart and graphs.
Arguments:
  df: airline dataframe
Returns:
  computed average dataframes for carrier delay, weather delay, NAS delay, and late aircraft delay
'''
def compute_data_choice_2(df):
  # compute delay average
  avg_car = df.groupby(['Month', 'Reporting_Airline'])['CarrierDelay'].mean().reset_index()
  avg_weather = df.groupby(['Month', 'Reporting_Airline'])['WeatherDelay'].mean().reset_index()
  avg_NAS = df.groupby(['Month', 'Reporting_Airline'])['NASDelay'].mean().reset_index()
  avg_sec = df.groupby(['Month', 'Reporting_Airline'])['SecurityDelay'].mean().reset_index()
  avg_late = df.groupby(['Month', 'Reporting_Airline'])['LateAircraftDelay'].mean().reset_index()
  return avg_car, avg_weather, avg_NAS, avg_sec, avg_late


# application layout
app.layout = html.Div(children = [# title of dash board
                                  html.H1('US Domestic Airline Flights Performance', 
                                          style={'textAlign':'center', 'color':'#503D36', 'font-size':24}),
                                  # outer division
                                  html.Div([#first inner division
                                            html.Div([# helper text for report type
                                                      html.Div([html.H2('Report Type:',
                                                                        style={'margin-right':'2em'}),]),
                                                      # report type dropdown
                                                      dcc.Dropdown(id='input-type',
                                                                   options=[{'label':'Yearly Airline Performance Report', 'value':'OPT1'},
                                                                            {'label': "Yearly Airline Delay Report", 'value':'OPT2'}],
                                                                   placeholder='Select a report type',
                                                                   style={'width':'80%', 'padding':'3px', 'font-size':'20px', 'textAlign':'center'})],
                                                     style={'display':'flex'}),
                                            #second inner division
                                            html.Div([# helper text for choosing year
                                                      html.Div([html.H2('Choose year:',
                                                                       style={'margin-right':'2em'}),]),
                                                      # year dropdown
                                                      dcc.Dropdown(id='input-year',
                                                                   options=[{'label':i, 'value':i} for i in year_list],
                                                                   placeholder='Select a year',
                                                                   style={'width':'80%', 'padding':'3px', 'font-size':'20px', 'textAlign':'center'})],
                                                     style={'display':'flex'})
                                            ]),
                                  # division for graph No1
                                  html.Div([ ], id='plot1'),
                                  # division for grsph No2, No3
                                  html.Div([html.Div([], id='plot2'), # div for graph No2
                                            html.Div([], id='plot3')],
                                            style={'display':'flex'}),# div for graph No3 
                                  # division for graph No4, No5
                                  html.Div([html.Div([], id='plot4'), # div for graph No4
                                            html.Div([], id='plot5')],
                                            style={'display':'flex'}) # div for graph No5
                                  ])


# callback function definition
@app.callback(#output
              [Output(component_id='plot1', component_property='children'),
               Output(component_id='plot2', component_property='children'),
               Output(component_id='plot3', component_property='children'),
               Output(component_id='plot4', component_property='children'),
               Output(component_id='plot5', component_property='children')],
              # input
              [Input(component_id='input-type', component_property='value'), 
               Input(component_id='input-year', component_property='value')],
              # holding output state till user enters all the form info
              [State('plot1', 'children'),
               State('plot2', 'children'),
               State('plot3', 'children'),
               State('plot4', 'children'),
               State('plot5', 'children')])

# add computation to callback function and return graphs
def get_graph(chart, year, children1, children2, c3, c4, c5):

  #select data
  df = airline_df[airline_df['Year']==int(year)]

  if chart == 'OPT1':
    # compute required data for creating graphs
    bar_data, line_data, pie_data, map_data, tree_data = compute_data_choice_1(df)
    # number of flights vs cancellation category
    bar_fig = px.bar(bar_data, x='Month', y='Flights', color='CancellationCode', title='Monthly flight cancellation')
    # evarage flight time by reporting airline
    line_fig = px.line(line_data, x='Month', y='AirTime', color='Reporting_Airline', title='Average monthly flight time (minutes) by reporting airline')
    # percentages of deverted airport landings
    pie_fig = px.pie(pie_data, values='Flights', names='Reporting_Airline', title='% of diverted airport landing by reporting airline')
    # map of flights from each state
    map_fig = px.choropleth(map_data,
                             locations='OriginState',
                             color='Flights',
                             hover_data=['OriginState', 'Flights'],
                             locationmode = 'USA-states', # set to plot as US States
                             color_continuous_scale = 'GnBu',
                             range_color=[0,map_data['Flights'].max()])
    map_fig.update_layout(title_text='Number of flights flying from each state', 
                          geo_scope='usa') #plot only usa instead of globe
    # number of flights flying to each state from each reporting airline
    tree_fig = px.treemap(tree_data,
                       path=['DestState', 'Reporting_Airline'],
                       values='Flights',
                       color='Flights',
                       color_continuous_scale='RdBu',
                       title='Flight count by airline to destination state')
    # return dcc.Graphs component to the empty divisions
    return [dcc.Graph(figure=tree_fig),
            dcc.Graph(figure=pie_fig),
            dcc.Graph(figure=map_fig),
            dcc.Graph(figure=bar_fig),
            dcc.Graph(figure=line_fig)]

  else:
    # compute required data for creating graphs
    avg_car, avg_weather, avg_NAS, avg_sec, avg_late = compute_data_choice_2(df)
    # monthly average carrier delay by reporting airline for the given year
    car_fig = px.line(avg_car, x='Month', y='CarrierDelay', color='Reporting_Airline', 
                          title='Monthly average carrier delay by reporting airline for the given year')
    # monthly average weather delay by reporting airline for the given year
    weather_fig = px.line(avg_weather, x='Month', y='WeatherDelay', color='Reporting_Airline', 
                          title='Monthly average weather delay by reporting airline for the given year')
    # monthly average NAS delay by reporting airline for the given year
    NAS_fig = px.line(avg_NAS, x='Month', y='NASDelay', color='Reporting_Airline', 
                          title='Monthly average NAS delay by reporting airline for the given year')
    # monthly average security delay by reporting airline for the given year
    sec_fig = px.line(avg_sec, x='Month', y='SecurityDelay', color='Reporting_Airline', 
                          title='Monthly average security delay by reporting airline for the given year')
    # monthly average late aircraft delay by reporting airline for the given year
    late_fig = px.line(avg_late, x='Month', y='LateAircraftDelay', color='Reporting_Airline', 
                          title='Monthly average late aircraft delay by reporting airline for the given year')
    # return dcc.Graphs component to the empty divisions
    return [dcc.Graph(figure=car_fig),
            dcc.Graph(figure=weather_fig),
            dcc.Graph(figure=NAS_fig),
            dcc.Graph(figure=sec_fig),
            dcc.Graph(figure=late_fig)]

# run the app
if __name__ == '__main__':
  app.run_server()


Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/



INFO:__main__:Dash is running on http://127.0.0.1:8050/



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [15/Feb/2023 01:09:41] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Feb/2023 01:09:41] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_8_1m1676422012.12.1.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Feb/2023 01:09:41] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_8_1m1676422012.14.0.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Feb/2023 01:09:41] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_8_1m1676422012.8.1.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Feb/2023 01:09:41] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_8_1m1676422012.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Feb/2023 01:09:41] "GET /_dash-component-suites/dash/dcc/dash_core_components-shared.v2_8_0m1676422012.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Feb/2023 01:09:41] "GET /_dash-component-suites/dash/dcc/dash

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.8/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/usr/local/lib/python3.8/dist-packages/dash/dash.py", line 1273, in dispatch
    ctx.run(
  File "/usr/local/lib/python3.8/

ERROR:__main__:Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.8/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/usr/local/lib/python3.8/dist-packages/dash/dash.py", line 1273, in dispatch
    ctx.run(
  File "/usr/local

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.8/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/usr/local/lib/python3.8/dist-packages/dash/dash.py", line 1273, in dispatch
    ctx.run(
  File "/usr/local/lib/python3.8/

ERROR:__main__:Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.8/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/usr/local/lib/python3.8/dist-packages/dash/dash.py", line 1273, in dispatch
    ctx.run(
  File "/usr/local

---

After running the application, follow the link created above to see the dashboard. Enjoy!

---